In [2]:
from keras.datasets import fashion_mnist
dataset= fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = dataset

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
print(X_train.shape)
print(y_train.shape)
 
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [29]:
sorted_arr = [-1] * 10

for x in range(10):
  boolean = 0
  sample = 0
  while(boolean == 0):
    if x == y_train[sample]:
      boolean = 1
      sorted_arr[x]=sample
    sample=sample+1
    
print(sorted_arr)


[1, 16, 5, 3, 19, 8, 18, 6, 23, 0]
